## Version Check

In [1]:
import sqlalchemy
sqlalchemy.__version__

'1.3.19'

## Connecting

In [2]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

## Declare a Mapping

In [3]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

In [4]:
from sqlalchemy import Column, Integer, String

class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String(50))
    fullname = Column(String(50))
    nickname = Column(String(50))

    def __repr__(self):
        return "<User(name='%s', fullname='%s', nickname='%s')>" % (
                            self.name, self.fullname, self.nickname)

## Create a Schema

In [5]:
User.__table__

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(length=50), table=<users>), Column('fullname', String(length=50), table=<users>), Column('nickname', String(length=50), table=<users>), schema=None)

In [6]:
Base.metadata.create_all(engine)

2021-02-07 12:33:51,946 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-02-07 12:33:51,948 INFO sqlalchemy.engine.base.Engine ()
2021-02-07 12:33:51,949 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-02-07 12:33:51,950 INFO sqlalchemy.engine.base.Engine ()
2021-02-07 12:33:51,951 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2021-02-07 12:33:51,952 INFO sqlalchemy.engine.base.Engine ()
2021-02-07 12:33:51,954 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("users")
2021-02-07 12:33:51,957 INFO sqlalchemy.engine.base.Engine ()
2021-02-07 12:33:51,959 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR(50), 
	fullname VARCHAR(50), 
	nickname VARCHAR(50), 
	PRIMARY KEY (id)
)


2021-02-07 12:33:51,960 INFO sqlalchemy.engine.base.Engine ()
2021-02-07 12:33:51,961 INFO sqlalchemy.engine.base.Engine COMMIT


## Create an Instance of the Mapped Class

In [7]:
ed_user = User(name='ed', fullname='Ed Jones', nickname='edsnickname')
ed_user.name

'ed'

In [8]:
ed_user.nickname

'edsnickname'

In [9]:
str(ed_user.id)

'None'

## Creating a Session¶

In [10]:
from sqlalchemy.orm import sessionmaker

In [11]:
Session = sessionmaker(bind=engine)

In [12]:
Session = sessionmaker()

In [13]:
Session.configure(bind=engine)

In [14]:
session = Session()

## Adding and Updating Objects

In [15]:
ed_user = User(name='ed', fullname='Ed Jones', nickname='edsnickname')
session.add(ed_user)

In [16]:
our_user = session.query(User).filter_by(name='ed').first() 
our_user

2021-02-07 12:36:02,123 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-07 12:36:02,125 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2021-02-07 12:36:02,126 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones', 'edsnickname')
2021-02-07 12:36:02,129 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2021-02-07 12:36:02,130 INFO sqlalchemy.engine.base.Engine ('ed', 1, 0)


<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>

In [17]:
ed_user is our_user

True

In [18]:
session.add_all([
    User(name='wendy', fullname='Wendy Williams', nickname='windy'),
    User(name='mary', fullname='Mary Contrary', nickname='mary'),
    User(name='fred', fullname='Fred Flintstone', nickname='freddy')])

In [19]:
ed_user.nickname = 'eddie'

In [20]:
session.dirty

IdentitySet([<User(name='ed', fullname='Ed Jones', nickname='eddie')>])

In [21]:
session.new

IdentitySet([<User(name='wendy', fullname='Wendy Williams', nickname='windy')>, <User(name='mary', fullname='Mary Contrary', nickname='mary')>, <User(name='fred', fullname='Fred Flintstone', nickname='freddy')>])

In [22]:
session.commit()

2021-02-07 12:39:15,836 INFO sqlalchemy.engine.base.Engine UPDATE users SET nickname=? WHERE users.id = ?
2021-02-07 12:39:15,837 INFO sqlalchemy.engine.base.Engine ('eddie', 1)
2021-02-07 12:39:15,839 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2021-02-07 12:39:15,839 INFO sqlalchemy.engine.base.Engine ('wendy', 'Wendy Williams', 'windy')
2021-02-07 12:39:15,841 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2021-02-07 12:39:15,841 INFO sqlalchemy.engine.base.Engine ('mary', 'Mary Contrary', 'mary')
2021-02-07 12:39:15,843 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2021-02-07 12:39:15,844 INFO sqlalchemy.engine.base.Engine ('fred', 'Fred Flintstone', 'freddy')
2021-02-07 12:39:15,846 INFO sqlalchemy.engine.base.Engine COMMIT


In [23]:
ed_user.id

2021-02-07 12:39:34,544 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-07 12:39:34,545 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2021-02-07 12:39:34,546 INFO sqlalchemy.engine.base.Engine (1,)


1

## Rolling Back

In [24]:
ed_user.name = 'Edwardo'

In [25]:
fake_user = User(name='fakeuser', fullname='Invalid', nickname='12345')
session.add(fake_user)

In [26]:
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

2021-02-07 12:42:11,261 INFO sqlalchemy.engine.base.Engine UPDATE users SET name=? WHERE users.id = ?
2021-02-07 12:42:11,262 INFO sqlalchemy.engine.base.Engine ('Edwardo', 1)
2021-02-07 12:42:11,264 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2021-02-07 12:42:11,264 INFO sqlalchemy.engine.base.Engine ('fakeuser', 'Invalid', '12345')
2021-02-07 12:42:11,266 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IN (?, ?)
2021-02-07 12:42:11,267 INFO sqlalchemy.engine.base.Engine ('Edwardo', 'fakeuser')


[<User(name='Edwardo', fullname='Ed Jones', nickname='eddie')>,
 <User(name='fakeuser', fullname='Invalid', nickname='12345')>]

In [27]:
session.rollback()

2021-02-07 12:43:01,691 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [28]:
ed_user.name

2021-02-07 12:43:08,923 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-07 12:43:08,925 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2021-02-07 12:43:08,926 INFO sqlalchemy.engine.base.Engine (1,)


'ed'

In [29]:
fake_user in session

False

In [30]:
session.query(User).filter(User.name.in_(['ed', 'fakeuser'])).all()

2021-02-07 12:43:40,873 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IN (?, ?)
2021-02-07 12:43:40,875 INFO sqlalchemy.engine.base.Engine ('ed', 'fakeuser')


[<User(name='ed', fullname='Ed Jones', nickname='eddie')>]

## Querying